<a href="https://colab.research.google.com/github/hatemamine/Improving-Arabic-IR-and-Reranking-Performance-with-Knowledge-Distillation/blob/main/eval_retrived_and_reranked_list_with_pyserini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# pyserini love java (install java)
!sudo apt-get update
!sudo apt-get install openjdk-21-jre -y
!java -version
!sudo  update-java-alternatives --list
!export JAVA_HOME='/usr/lib/jvm/java-1.21.0-openjdk-amd64'
import os
# linux
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-1.21.0-openjdk-amd64"
# installing pyserini
#https://github.com/castorini/pyserini
!python --version
!pip install faiss-gpu -q
!pip install pyserini -q
# install numpy==1.26.4 to fix package conflit
import numpy
numpy.__version__
#!pip show numpy
!pip install --upgrade numpy==1.26.4

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,312 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,673 kB]
Get:13 https://r2u.stat.illinois.edu/ubu

In [2]:
# 100 bm25 mr.Tydi
!python -m pyserini.search.lucene \
  --threads 16 --batch-size 128 \
  --language ar \
  --topics mrtydi-v1.1-arabic-test \
  --index mrtydi-v1.1-ar \
  --output run.mrtydi.bm25.ar.test.txt --bm25 --hits 1000
!python -m pyserini.eval.trec_eval \
  -c -m recip_rank.100 -m recall.10,100 mrtydi-v1.1-arabic-test \
  run.mrtydi.bm25.ar.test.txt
!python -m pyserini.eval.trec_eval \
-c -m recip_rank.100 -m recall.10,1000 mrtydi-v1.1-arabic-test \
  run.mrtydi.bm25.ar.test.txt

2025-02-13 10:17:15.145184: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739441835.541097    2088 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739441835.646760    2088 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-13 10:17:16.488981: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-13 10:17:26,070 INFO  [main] topicreader.TopicReader (TopicReader.java:229) - Downloading topics from https://raw.git

#zero-shot evaluation
#mr.tydi
https://castorini.github.io/pyserini/2cr/mrtydi.html
https://github.com/castorini/mr.tydi

In [ ]:
from huggingface_hub import hf_hub_download
organization_dataset_id= "hatemestinbejaia/ExperimentDATA_knowledge_distillation_vs_fine_tuning"
#zero-shot evaluation reranked bm25-100 mr.Tydi
hf_hub_download(repo_id=organization_dataset_id, filename="qrels.test.mrtydi.txt", local_dir="./", repo_type="dataset")
ranked_list_mrTydi=["mrtydi_reranked/mmarco-Arabic-mMiniLML-cross-encoder-KD-v1.tsvrun.mrtydi.bm25.ar.test.txt", "mrtydi_reranked/mmarco-Arabic-mMiniLML-cross-encoder-NoKD-v1.tsvrun.mrtydi.bm25.ar.test.txt", "mrtydi_reranked/mmarco-Arabic-AraDPR-cross-encoder-KD-v1_run.mrtydi.bm25.ar.test.txt", "mrtydi_reranked/mmarco-Arabic-AraDPR-cross-encoder-NoKD-v1_run.mrtydi.bm25.ar.test.txt", "mrtydi_reranked/mmarco-Arabic-AraElectra-cross-encoder-KD-v1_run.mrtydi.bm25.ar.test.txt", "mrtydi_reranked/mmarco-Arabic-AraElectra-cross-encoder-NoKD-v1_run.mrtydi.bm25.ar.test.txt" ]
for i in range(len(ranked_list_mrTydi)):
    hf_hub_download(repo_id=organization_dataset_id, filename=ranked_list_mrTydi[i], local_dir="./", repo_type="dataset")
for i in ranked_list_mrTydi:
        print("*****************************************")
        print(i)
        print("*****************************************")
        !python -m pyserini.eval.trec_eval  -c -m recip_rank.100 -m recall.100\
            qrels.test.mrtydi.txt \
            {i}
        #break

qrels.test.mrtydi.txt:   0%|          | 0.00/24.9k [00:00<?, ?B/s]

(…)der-KD-v1.tsvrun.mrtydi.bm25.ar.test.txt:   0%|          | 0.00/1.98M [00:00<?, ?B/s]

(…)r-NoKD-v1.tsvrun.mrtydi.bm25.ar.test.txt:   0%|          | 0.00/1.98M [00:00<?, ?B/s]

(…)ncoder-KD-v1_run.mrtydi.bm25.ar.test.txt:   0%|          | 0.00/1.98M [00:00<?, ?B/s]

(…)oder-NoKD-v1_run.mrtydi.bm25.ar.test.txt:   0%|          | 0.00/1.98M [00:00<?, ?B/s]

(…)ncoder-KD-v1_run.mrtydi.bm25.ar.test.txt:   0%|          | 0.00/1.98M [00:00<?, ?B/s]

(…)oder-NoKD-v1_run.mrtydi.bm25.ar.test.txt:   0%|          | 0.00/1.98M [00:00<?, ?B/s]

*****************************************
mrtydi_reranked/mmarco-Arabic-mMiniLML-cross-encoder-KD-v1.tsvrun.mrtydi.bm25.ar.test.txt
*****************************************
msmarco run detected. Converting to trec...
recip_rank            	all	0.5149
recall_100            	all	0.7928
*****************************************
mrtydi_reranked/mmarco-Arabic-mMiniLML-cross-encoder-NoKD-v1.tsvrun.mrtydi.bm25.ar.test.txt
*****************************************
msmarco run detected. Converting to trec...
recip_rank            	all	0.4927
recall_100            	all	0.7928
*****************************************
mrtydi_reranked/mmarco-Arabic-AraDPR-cross-encoder-KD-v1_run.mrtydi.bm25.ar.test.txt
*****************************************
msmarco run detected. Converting to trec...
recip_rank            	all	0.5664
recall_100            	all	0.7928
*****************************************
mrtydi_reranked/mmarco-Arabic-AraDPR-cross-encoder-NoKD-v1_run.mrtydi.bm25.ar.test.txt
****************

In [ ]:
from huggingface_hub import hf_hub_download
#zero-shot evaluation first stage mr.Tydi
hf_hub_download(repo_id=organization_dataset_id, filename="qrels.test.mrtydi.txt", local_dir="./", repo_type="dataset")
ranked_list_mrTydi=["FirstStage_mrTydi/FirstStagemrTydi-collection-arabic-embedding-KDaradpr.tsv", "FirstStage_mrTydi/FirstStagemrTydi-collection-arabic-embedding-NOKDaradpr.tsv", "FirstStage_mrTydi/FirstStagemrTydi-collection-arabic-embedding-mmarco-Arabic-AraElectra-bi-encoder-KD-v1.tsv", "FirstStage_mrTydi/FirstStagemrTydi-collection-arabic-embedding-mmarco-Arabic-AraElectra-bi-encoder-NoKD-v1.tsv", "FirstStage_mrTydi/FirstStagemrTydi-collection-arabic-embedding-mmarco-Arabic-mMiniLML-bi-encoder-KD-v1.tsv", "FirstStage_mrTydi/FirstStagemrTydi-collection-arabic-embedding-mmarco-Arabic-mMiniLML-bi-encoder-NoKD-v1.tsv", "FirstStagecombine.tsv", "FirstStagecombine1000.tsv", "mmarco-Arabic-AraElectra-cross-encoder-KD-v1_FirstStagecombine1000.tsv", "mmarco-Arabic-AraElectra-cross-encoder-KD-v1_run.mrtydi.bm25.ar.test1000.txt" ]
for i in range(len(ranked_list_mrTydi)):
    hf_hub_download(repo_id=organization_dataset_id, filename=ranked_list_mrTydi[i], local_dir="./", repo_type="dataset")
for i in ranked_list_mrTydi:
        print("*****************************************")
        print(i)
        print("*****************************************")
        !python -m pyserini.eval.trec_eval   -m  recip_rank.100 -m recall.100 \
            qrels.test.mrtydi.txt \
            {i}
        #break

(…)collection-arabic-embedding-KDaradpr.tsv:   0%|          | 0.00/21.2M [00:00<?, ?B/s]

(…)llection-arabic-embedding-NOKDaradpr.tsv:   0%|          | 0.00/2.02M [00:00<?, ?B/s]

(…)o-Arabic-AraElectra-bi-encoder-KD-v1.tsv:   0%|          | 0.00/2.01M [00:00<?, ?B/s]

(…)Arabic-AraElectra-bi-encoder-NoKD-v1.tsv:   0%|          | 0.00/2.01M [00:00<?, ?B/s]

(…)rco-Arabic-mMiniLML-bi-encoder-KD-v1.tsv:   0%|          | 0.00/2.01M [00:00<?, ?B/s]

...

FirstStagecombine.tsv:   0%|          | 0.00/2.01M [00:00<?, ?B/s]

FirstStagecombine1000.tsv:   0%|          | 0.00/21.2M [00:00<?, ?B/s]

(…)-encoder-KD-v1_FirstStagecombine1000.tsv:   0%|          | 0.00/21.2M [00:00<?, ?B/s]

(…)er-KD-v1_run.mrtydi.bm25.ar.test1000.txt:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

*****************************************
mmarco-Arabic-AraElectra-cross-encoder-KD-v1_run.mrtydi.bm25.ar.test1000.txt
*****************************************
msmarco run detected. Converting to trec...
recip_rank            	all	0.5755
recall_100            	all	0.8915


In [ ]:
# evaluate first stage mmarco
qerl_file ="qrels.msmarco-passage.dev-subset.txt"
ranked_list=["FirstStage_mmarco/FirstStage-mmarco-Arabic-mMiniLML-bi-encoder-KD-v1.tsv", "FirstStage_mmarco/FirstStage-mmarco-Arabic-mMiniLML-bi-encoder-NoKD-v1.tsv", "FirstStage_mmarco/FirstStage-mmarco-Arabic-AraDPR-bi-encoder-KD-v1.tsv", "FirstStage_mmarco/FirstStage-mmarco-Arabic-AraDPR-bi-encoder-NoKD-v1.tsv", "FirstStage_mmarco/FirstStage-mmarco-Arabic-AraElectra-bi-encoder-KD-v1.tsv", "FirstStage_mmarco/FirstStage-mmarco-Arabic-AraElectra-bi-encoder-NoKD-v1.tsv", "FirstStageArabicmmarcopassageAraDPRetmMiniLM.dev.small.tsv" ]
from huggingface_hub import hf_hub_download
hf_hub_download(repo_id=organization_dataset_id, filename=qerl_file, local_dir="./", repo_type="dataset")
for i in range(len(ranked_list)):
    hf_hub_download(repo_id=organization_dataset_id, filename=ranked_list[i], local_dir="./", repo_type="dataset")
for i in ranked_list:
        print("*****************************************")
        print(i)
        print("*****************************************")
        #!python -m pyserini.eval.msmarco_passage_eval \ # to get only MRR@10
        !python -m pyserini.eval.trec_eval  -c -m recip_rank.10 -m recall.1000\
            qrels.msmarco-passage.dev-subset.txt \
            {i}
        #break

qrels.msmarco-passage.dev-subset.txt:   0%|          | 0.00/143k [00:00<?, ?B/s]

(…)rco-Arabic-mMiniLML-bi-encoder-KD-v1.tsv:   0%|          | 0.00/133M [00:00<?, ?B/s]

(…)o-Arabic-mMiniLML-bi-encoder-NoKD-v1.tsv:   0%|          | 0.00/133M [00:00<?, ?B/s]

(…)marco-Arabic-AraDPR-bi-encoder-KD-v1.tsv:   0%|          | 0.00/133M [00:00<?, ?B/s]

(…)rco-Arabic-AraDPR-bi-encoder-NoKD-v1.tsv:   0%|          | 0.00/133M [00:00<?, ?B/s]

(…)o-Arabic-AraElectra-bi-encoder-KD-v1.tsv:   0%|          | 0.00/133M [00:00<?, ?B/s]

(…)Arabic-AraElectra-bi-encoder-NoKD-v1.tsv:   0%|          | 0.00/133M [00:00<?, ?B/s]

(…)arcopassageAraDPRetmMiniLM.dev.small.tsv:   0%|          | 0.00/133M [00:00<?, ?B/s]

*****************************************
FirstStage_mmarco/FirstStage-mmarco-Arabic-mMiniLML-bi-encoder-KD-v1.tsv
*****************************************
msmarco run detected. Converting to trec...
recip_rank            	all	0.1839
recall_1000           	all	0.7771


In [ ]:
# evaluate second stage Arabic mMARCO reranked 1000bm25
qerl_file ="qrels.msmarco-passage.dev-subset.txt"
ranked_list=["mmarco_run1000bm25Reranked/mmarco-Arabic-mMiniLML-cross-encoder-KD-v1.tsv", "mmarco_run1000bm25Reranked/mmarco-Arabic-mMiniLML-cross-encoder-NoKD-v1.tsv", "mmarco_run1000bm25Reranked/mmarco-Arabic-AraDPR-cross-encoder-KD-v1.tsv", "mmarco_run1000bm25Reranked/mmarco-Arabic-AraDPR-cross-encoder-NoKD-v1.tsv", "mmarco_run1000bm25Reranked/mmarco-Arabic-AraElectra-cross-encoder-KD-v1.tsv", "mmarco_run1000bm25Reranked/mmarco-Arabic-AraElectra-cross-encoder-NoKD-v1.tsv" ]
from huggingface_hub import hf_hub_download
hf_hub_download(repo_id=organization_dataset_id, filename=qerl_file, local_dir="./", repo_type="dataset")
for i in range(len(ranked_list)):
    hf_hub_download(repo_id=organization_dataset_id, filename=ranked_list[i], local_dir="./", repo_type="dataset")
for i in ranked_list:
        print("*****************************************")
        print(i)
        print("*****************************************")
        #!python -m pyserini.eval.msmarco_passage_eval \ # to get only MRR@10
        !python -m pyserini.eval.trec_eval  -c  -m recip_rank.10 -m recall.1000\
            qrels.msmarco-passage.dev-subset.txt \
            {i}
        #break

(…)-Arabic-mMiniLML-cross-encoder-KD-v1.tsv:   0%|          | 0.00/131M [00:00<?, ?B/s]

(…)rabic-mMiniLML-cross-encoder-NoKD-v1.tsv:   0%|          | 0.00/131M [00:00<?, ?B/s]

(…)co-Arabic-AraDPR-cross-encoder-KD-v1.tsv:   0%|          | 0.00/131M [00:00<?, ?B/s]

(…)-Arabic-AraDPR-cross-encoder-NoKD-v1.tsv:   0%|          | 0.00/131M [00:00<?, ?B/s]

(…)rabic-AraElectra-cross-encoder-KD-v1.tsv:   0%|          | 0.00/131M [00:00<?, ?B/s]

(…)bic-AraElectra-cross-encoder-NoKD-v1.tsv:   0%|          | 0.00/131M [00:00<?, ?B/s]

*****************************************
mmarco_run1000bm25Reranked/mmarco-Arabic-mMiniLML-cross-encoder-KD-v1.tsv
*****************************************
msmarco run detected. Converting to trec...
recip_rank            	all	0.2356
recall_1000           	all	0.6369
